# **과제2 : 챗봇 모델 ?��?��?���?**

## **1.?��경�??�?**

### (1) ?��?��브러�? Import

In [1]:
import pandas as pd
import numpy as np
import os
import sqlite3
import openai

from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, Document
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA, ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

### (2) OpenAI API Key ?��?��
* ?��경�???���? ?��록된 Key ?��?��?���?

In [ ]:
# ?��경�???��?��?�� ?�� 불러?���?
api_key = os.getenv('OPENAI_API_KEY')
print(api_key) 

* 만약 ?��경�???�� ?�� ?��?��?�� ?�� ?��?��?���? ?��?�� 코드????�� 주석?�� ?��?��?���?, ?��?��?�� api key�? ?��?��?���? ?��?��
    * ?��?�� 코드?�� ?�� �??��?�� **?��?��**�? ?��?��?��.
    * ?��?��?�� ?��?��(.ipynb, .py)?��?��?�� 매번 ?��?��?��?�� ?��.

In [62]:
# os.environ['OPENAI_API_KEY'] = '?��?��분의 OpenAI API?��' 
# openai.api_key = os.getenv('OPENAI_API_KEY')

## **2.Vector DB 만들�?**

* ?��?��?�� 로딩 
    * 1?��차에?�� ?��공한 ?��?��?�� 구조�? 그�??�? ?��?��
    * ?��?��블스�? ?��?��?���? FAQ ?��?��?�� ?���?(https://aivle.kt.co.kr/home/brd/faq/main?mcd=MC00000056)
        * 모든 질문?�� csv ?��?���? ????��
    * ?��?��?��?��?��?��?���? ????��?���?

In [ ]:
data = pd.read_csv('aivleschool_qa.csv', encoding='utf-8')
data.head()

* 벡터 ?��?��?��베이?��
    * 1?���? 벡터 ?��?��?��베이?���? 그�??�? ?��?��
        * Embedding 모델 : text-embedding-ada-002
        * DB 경로 : ./database



In [2]:
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
database = Chroma(persist_directory="./database", embedding_function = embeddings )

* ?��?��?�� ?��?��
    * ?���? ?��?��?�� ?��?��?��?�� ?��?��?���? ?��머�?? ?��?��?���?

In [ ]:
# �? ?��?�� ?��?��?���? Document 객체�? �??�� 
documents = [Document(page_content=text) for text in data['QA'].tolist()]

# ?��?��?��?��?��?��?��?�� 문서 추�??
database.add_documents(documents)

* ?��?��?�� ?��?��?�� 조회

In [ ]:
database.get()

## **3.RAG+LLM모델**

* 모델 ?��?�� : ConversationalRetrievalChain
    * LLM 모델 : gpt-3.5-turbo
    * retriever : 벡터DB
        * ?��?��?�� ?��??? 문서 5�? �??��?��?���? ?��?��
    * memory
* 결과
    * ?��?��?�� ?��??? 문장 3개�?? ?��?��?�� ?��?��?�� ?���? 출력?��?���? 코드 ?��?��
    * 질문 history �?리�?? ?��?�� ?��?�� ????��
        * timestamp
        * 질문
        * ?��?��?�� 3�?
        * ?���?
        * 그외


In [ ]:
chat = ChatOpenAI(model="gpt-3.5-turbo")

k=3
retriever = database.as_retriever(search_kwargs={"k": k})

# ????�� 메모�? ?��?��
memory = ConversationBufferMemory(memory_key="chat_history", input_key="question", output_key="answer", 
                                  return_messages=True)

# ConversationalRetrievalQA 체인 ?��?��
qa = ConversationalRetrievalChain.from_llm(llm=chat, retriever=retriever, memory=memory, 
                                           return_source_documents=True,  output_key="answer")

* 모델 ?��?��?���?
    * ?��?��블스�? �??�� ?���? �??�� 질문?�� ?���? ?��?�� ?��?��?�� 봅시?��.

In [ ]:
query = '�??��?��?��?�� ?��?�� ?��?��?�� ?��?��?��?'
result = qa(query)

print(result["answer"])